In [37]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# 1. 데이터 불러오기
customers = pd.read_csv('./olist_customers_dataset.csv')
sellers = pd.read_csv('./olist_sellers_dataset.csv')
geolocation = pd.read_csv('./olist_geolocation_dataset.csv')
orders = pd.read_csv('./olist_orders_dataset.csv')
order_items = pd.read_csv('./olist_order_items_dataset.csv')
payments = pd.read_csv('./olist_order_payments_dataset.csv')
reviews = pd.read_csv('./olist_order_reviews_dataset.csv')
products = pd.read_csv('./olist_products_dataset.csv')
category_translation = pd.read_csv('./product_category_name_translation.csv')

# 2. 우편번호별 위경도 평균값으로 공간정보 집계
geo_agg = geolocation.groupby('geolocation_zip_code_prefix').agg({
    'geolocation_lat': 'mean',
    'geolocation_lng': 'mean'
}).reset_index()

# 3. customers에 공간정보 병합 및 컬럼명 변경
customers_geo = customers.merge(
    geo_agg,
    left_on='customer_zip_code_prefix',
    right_on='geolocation_zip_code_prefix',
    how='left'
).rename(columns={
    'geolocation_lat': 'customer_lat',
    'geolocation_lng': 'customer_lng'
})

# 4. sellers에 공간정보 병합 및 컬럼명 변경
sellers_geo = sellers.merge(
    geo_agg,
    left_on='seller_zip_code_prefix',
    right_on='geolocation_zip_code_prefix',
    how='left'
).rename(columns={
    'geolocation_lat': 'seller_lat',
    'geolocation_lng': 'seller_lng'
})

# 5. 순차 병합
df = (
    orders
    .merge(customers_geo, on='customer_id', how='left')
    .merge(order_items, on='order_id', how='left')
    .merge(products, on='product_id', how='left')
    .merge(sellers_geo, on='seller_id', how='left')
    .merge(payments, on='order_id', how='left')
    .merge(reviews, on='order_id', how='left')
    .merge(category_translation, on='product_category_name', how='left')
)

# 6. 실제 컬럼명 확인 (이 부분에서 꼭 확인!)
print("최종 컬럼명:", df.columns.tolist())

# 7. 고객 기준 geometry, WKT 생성
df['geometry'] = df.apply(
    lambda row: Point(row['customer_lng'], row['customer_lat']) if pd.notnull(row['customer_lat']) and pd.notnull(row['customer_lng']) else None,
    axis=1
)
df['geometry_wkt'] = df['geometry'].apply(lambda x: x.wkt if x is not None else None)

# 8. 판매자 기준 geometry, WKT 생성 (선택)
df['seller_geometry'] = df.apply(
    lambda row: Point(row['seller_lng'], row['seller_lat']) if pd.notnull(row['seller_lat']) and pd.notnull(row['seller_lng']) else None,
    axis=1
)
df['seller_geometry_wkt'] = df['seller_geometry'].apply(lambda x: x.wkt if x is not None else None)

# 9. GeoDataFrame으로 변환 (geometry=고객 기준)
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:4326")

# 10. 파일로 저장 (CSV + GeoPackage)
gdf.to_file('./olist_full_merged.gpkg', driver='GPKG')
df.to_csv('./olist_full_merged.csv', index=False)

# 11. 미리보기
print(gdf[['customer_id', 'customer_lat', 'customer_lng', 'seller_id', 'seller_lat', 'seller_lng', 'geometry', 'geometry_wkt', 'seller_geometry', 'seller_geometry_wkt']].head())

최종 컬럼명: ['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date', 'customer_unique_id', 'customer_zip_code_prefix', 'customer_city', 'customer_state', 'geolocation_zip_code_prefix_x', 'customer_lat', 'customer_lng', 'order_item_id', 'product_id', 'seller_id', 'shipping_limit_date', 'price', 'freight_value', 'product_category_name', 'product_name_lenght', 'product_description_lenght', 'product_photos_qty', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm', 'seller_zip_code_prefix', 'seller_city', 'seller_state', 'geolocation_zip_code_prefix_y', 'seller_lat', 'seller_lng', 'payment_sequential', 'payment_type', 'payment_installments', 'payment_value', 'review_id', 'review_score', 'review_comment_title', 'review_comment_message', 'review_creation_date', 'review_answer_timestamp', 'product_category_name_english']
                

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119143 entries, 0 to 119142
Data columns (total 50 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   order_id                       119143 non-null  object 
 1   customer_id                    119143 non-null  object 
 2   order_status                   119143 non-null  object 
 3   order_purchase_timestamp       119143 non-null  object 
 4   order_approved_at              118966 non-null  object 
 5   order_delivered_carrier_date   117057 non-null  object 
 6   order_delivered_customer_date  115722 non-null  object 
 7   order_estimated_delivery_date  119143 non-null  object 
 8   customer_unique_id             119143 non-null  object 
 9   customer_zip_code_prefix       119143 non-null  int64  
 10  customer_city                  119143 non-null  object 
 11  customer_state                 119143 non-null  object 
 12  geolocation_zip_code_prefix_x 

In [39]:
df.isnull().sum()

order_id                              0
customer_id                           0
order_status                          0
order_purchase_timestamp              0
order_approved_at                   177
order_delivered_carrier_date       2086
order_delivered_customer_date      3421
order_estimated_delivery_date         0
customer_unique_id                    0
customer_zip_code_prefix              0
customer_city                         0
customer_state                        0
geolocation_zip_code_prefix_x       322
customer_lat                        322
customer_lng                        322
order_item_id                       833
product_id                          833
seller_id                           833
shipping_limit_date                 833
price                               833
freight_value                       833
product_category_name              2542
product_name_lenght                2542
product_description_lenght         2542
product_photos_qty                 2542


In [40]:
df.describe(include='all')

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,...,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,product_category_name_english,geometry,geometry_wkt,seller_geometry,seller_geometry_wkt
count,119143,119143,119143,119143,118966,117057,115722,119143,119143,119143.000000,...,118146.000000,13989,50245,118146,118146,116576,118821,118821,118045,118045
unique,99441,99441,8,98875,90733,81018,95664,459,96096,NaN,...,NaN,4527,36159,636,98248,71,14837,14837,2239,2239
top,895ab968e7bb0d5659d16cd74cd1650c,270c23a11d024a44c896d1894b261a83,delivered,2017-08-08 20:26:31,2017-08-08 20:43:31,2017-08-10 11:58:14,2017-08-14 12:46:18,2017-12-20 00:00:00,9a736b248f67d166d2fbb006bcb877c3,NaN,...,NaN,Recomendo,Muito bom,2017-12-19 00:00:00,2017-08-17 22:17:55,bed_bath_table,POINT (-43.107023598220486 -22.90324481661885),POINT (-43.107023598220486 -22.90324481661885),POINT (-48.82974353343716 -21.75732063771276),POINT (-48.82974353343716 -21.75732063771276)
freq,63,63,115723,63,63,63,63,663,75,NaN,...,NaN,494,259,547,63,11988,164,164,8373,8373
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35033.451298,...,4.015582,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29823.198969,...,1.400436,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1003.000000,...,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11250.000000,...,4.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24240.000000,...,5.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58475.000000,...,5.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
column_rename_map = {
    # 주문 항목 정보
    'order_id': '주문 ID',
    'order_item_id': '주문 내 항목 순번',
    'product_id': '상품 ID',
    'seller_id': '판매자 ID',
    'shipping_limit_date': '배송 마감 기한',
    'price': '상품 가격',
    'freight_value': '배송비',

    # 결제 정보
    'payment_sequential': '결제 순번',
    'payment_type': '결제 방식',
    'payment_installments': '할부 개월 수',
    'payment_value': '결제 금액',

    # 리뷰 정보
    'review_id': '리뷰 ID',
    'review_score': '리뷰 점수',
    'review_comment_title': '리뷰 제목',
    'review_comment_message': '리뷰 내용',
    'review_creation_date': '리뷰 작성일',
    'review_answer_timestamp': '리뷰 응답 시각',

    # 지역 정보
    'geolocation_zip_code_prefix_x': '고객 우편번호 앞자리',
    'customer_lat': '고객 위도',
    'customer_lng': '고객 경도',
    'geolocation_city': '도시명',
    'geolocation_state': '주(State) 코드',
    'geolocation_zip_code_prefix_y': '지리정보 우편번호 앞자리',
    'geometry': '고객 공간 좌표',
    'geometry_wkt': '고객 공간 좌표 문자',

    # 주문 기본 정보
    'customer_id': '고객 ID',
    'order_status': '주문 상태',
    'order_purchase_timestamp': '주문 시간',
    'order_approved_at': '결제 승인 시각',
    'order_delivered_carrier_date': '택배사 배송 시작일',
    'order_delivered_customer_date': '고객 수령일 (실제 배송일)',
    'order_estimated_delivery_date': '예상 배송일',

    # 고객 정보
    'customer_unique_id': '고유 고객 ID',
    'customer_zip_code_prefix': '고객 우편번호 앞자리',
    'customer_city': '고객 도시',
    'customer_state': '고객 주(State)',

    # 제품 정보
    'product_category_name': '제품카테고리명',
    'product_name_lenght': '제품명 길이',
    'product_description_lenght': '제품 설명 길이',
    'product_photos_qty': '제품 사진 개수',
    'product_weight_g': '제품 무게',
    'product_length_cm': '제품 길이',
    'product_height_cm': '제품 높이',
    'product_width_cm': '제품 너비',

    # 판매자 정보
    'seller_zip_code_prefix': '판매자 우편번호 앞자리',
    'seller_city': '판매자 도시',
    'seller_state': '판매자 주',
    'seller_lat': '판매자 위도',
    'seller_lng': '판매자 경도',

    # 카테고리 번역
    'product_category_name_english': '제품 카테고리명 영어',

    # 추가: 판매자 공간좌표
    'seller_geometry': '판매자 공간 좌표',
    'seller_geometry_wkt': '판매자 공간 좌표 문자',
}

In [47]:
df1=df.copy()
df1= df1.rename(columns = column_rename_map)
df1

,주문 ID,고객 ID,주문 상태,주문 시간,결제 승인 시각,택배사 배송 시작일,고객 수령일 (실제 배송일),예상 배송일,고유 고객 ID,고객 우편번호 앞자리,...,리뷰 점수,리뷰 제목,리뷰 내용,리뷰 작성일,리뷰 응답 시각,제품 카테고리명 영어,고객 공간 좌표,고객 공간 좌표 문자,판매자 공간 좌표,판매자 공간 좌표 문자
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,...,4.0,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48,housewares,POINT (-46.587161274276774 -23.57698293467452),POINT (-46.587161274276774 -23.57698293467452),POINT (-46.444238272702385 -23.680729252163324),POINT (-46.444238272702385 -23.680729252163324)
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,...,4.0,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48,housewares,POINT (-46.587161274276774 -23.57698293467452),POINT (-46.587161274276774 -23.57698293467452),POINT (-46.444238272702385 -23.680729252163324),POINT (-46.444238272702385 -23.680729252163324)
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,...,4.0,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48,housewares,POINT (-46.587161274276774 -23.57698293467452),POINT (-46.587161274276774 -23.57698293467452),POINT (-46.444238272702385 -23.680729252163324),POINT (-46.444238272702385 -23.680729252163324)
3,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,af07308b275d755c9edb36a90c618231,47813,...,4.0,Muito boa a loja,Muito bom o produto.,2018-08-08 00:00:00,2018-08-08 18:37:50,perfumery,POINT (-44.660710516494206 -12.177924305463309),POINT (-44.660710516494206 -12.177924305463309),POINT (-43.98042684085744 -19.80768065786215),POINT (-43.98042684085744 -19.80768065786215)
4,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,3a653a41f6f9fc3d2a113cf8398680e8,75265,...,5.0,NaN,NaN,2018-08-18 00:00:00,2018-08-22 19:07:58,auto,POINT (-48.51478334802312 -16.745149541882594),POINT (-48.51478334802312 -16.745149541882594),POINT (-48.22960077888057 -21.36350170979939),POINT (-48.22960077888057 -21.36350170979939)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119138,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02 00:00:00,da62f9e57a76d978d02ab5362c509660,11722,...,4.0,NaN,So uma peça que veio rachado mas tudo bem rs,2018-03-01 00:00:00,2018-03-02 17:50:01,baby,POINT (-46.4498635167305 -24.001499900399846),POINT (-46.4498635167305 -24.001499900399846),POINT (-50.49834751389763 -21.930547778284403),POINT (-50.49834751389763 -21.930547778284403)
119139,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27 00:00:00,737520a9aad80b3fbbdad19b66b37b30,45920,...,5.0,NaN,Foi entregue antes do prazo.,2017-09-22 00:00:00,2017-09-22 23:10:57,home_appliances_2,POINT (-39.37363049842613 -17.898358182362454),POINT (-39.37363049842613 -17.898358182362454),POINT (-46.452661287455555 -23.553641778735507),POINT (-46.452661287455555 -23.553641778735507)
119140,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:

In [48]:
df1.columns

Index(['주문 ID', '고객 ID', '주문 상태', '주문 시간', '결제 승인 시각', '택배사 배송 시작일',
       '고객 수령일 (실제 배송일)', '예상 배송일', '고유 고객 ID', '고객 우편번호 앞자리', '고객 도시',
       '고객 주(State)', '고객 우편번호 앞자리', '고객 위도', '고객 경도', '주문 내 항목 순번', '상품 ID',
       '판매자 ID', '배송 마감 기한', '상품 가격', '배송비', '제품카테고리명', '제품명 길이', '제품 설명 길이',
       '제품 사진 개수', '제품 무게', '제품 길이', '제품 높이', '제품 너비', '판매자 우편번호 앞자리',
       '판매자 도시', '판매자 주', '지리정보 우편번호 앞자리', '판매자 위도', '판매자 경도', '결제 순번',
       '결제 방식', '할부 개월 수', '결제 금액', '리뷰 ID', '리뷰 점수', '리뷰 제목', '리뷰 내용',
       '리뷰 작성일', '리뷰 응답 시각', '제품 카테고리명 영어', '고객 공간 좌표', '고객 공간 좌표 문자',
       '판매자 공간 좌표', '판매자 공간 좌표 문자'],
      dtype='object')

In [49]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119143 entries, 0 to 119142
Data columns (total 50 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   주문 ID            119143 non-null  object 
 1   고객 ID            119143 non-null  object 
 2   주문 상태            119143 non-null  object 
 3   주문 시간            119143 non-null  object 
 4   결제 승인 시각         118966 non-null  object 
 5   택배사 배송 시작일       117057 non-null  object 
 6   고객 수령일 (실제 배송일)  115722 non-null  object 
 7   예상 배송일           119143 non-null  object 
 8   고유 고객 ID         119143 non-null  object 
 9   고객 우편번호 앞자리      119143 non-null  int64  
 10  고객 도시            119143 non-null  object 
 11  고객 주(State)      119143 non-null  object 
 12  고객 우편번호 앞자리      118821 non-null  float64
 13  고객 위도            118821 non-null  float64
 14  고객 경도            118821 non-null  float64
 15  주문 내 항목 순번       118310 non-null  float64
 16  상품 ID            118310 non-null  obje

In [52]:
drop_cols = [
    # 기존
    "결제 방식",           # payment_type
    "할부 개월 수",         # payment_installments
    "상품 ID",             # product_id
    "제품카테고리명",        # product_category_name
    "제품명 길이",          # product_name_lenght
    "제품 설명 길이",        # product_description_lenght
    "제품 사진 개수",        # product_photos_qty
    "제품 무게",            # product_weight_g
    "제품 길이",            # product_length_cm
    "제품 높이",            # product_height_cm
    "제품 너비",            # product_width_cm
    "택배사 배송 시작일",     # order_delivered_carrier_date
    "제품 카테고리명 영어",   # product_category_name_english
    "지리정보 우편번호 앞자리", # geolocation_zip_code_prefix_y
    "고객 공간 좌표 문자",         # geometry_wkt
    "판매자 공간 좌표 문자",    # seller_geometry_wkt
]

In [53]:
df1.drop(drop_cols, axis=1,  inplace=True)

In [54]:
for col in df1.columns:
    # 만약 컬럼명이 중복이면 DataFrame이 반환, 아닐 땐 Series
    # 그래서 DataFrame이면 첫 번째 컬럼만 사용
    s = df1[col]
    if isinstance(s, pd.DataFrame):
        s = s.iloc[:, 0]
    nan_count = s.isnull().sum()
    nan_ratio = nan_count / len(df1) * 100
    print(f'{col}: 결측치 {nan_count}개, 결측치 비율 {nan_ratio:.2f}%')

주문 ID: 결측치 0개, 결측치 비율 0.00%
고객 ID: 결측치 0개, 결측치 비율 0.00%
주문 상태: 결측치 0개, 결측치 비율 0.00%
주문 시간: 결측치 0개, 결측치 비율 0.00%
결제 승인 시각: 결측치 177개, 결측치 비율 0.15%
고객 수령일 (실제 배송일): 결측치 3421개, 결측치 비율 2.87%
예상 배송일: 결측치 0개, 결측치 비율 0.00%
고유 고객 ID: 결측치 0개, 결측치 비율 0.00%
고객 우편번호 앞자리: 결측치 0개, 결측치 비율 0.00%
고객 도시: 결측치 0개, 결측치 비율 0.00%
고객 주(State): 결측치 0개, 결측치 비율 0.00%
고객 우편번호 앞자리: 결측치 0개, 결측치 비율 0.00%
고객 위도: 결측치 322개, 결측치 비율 0.27%
고객 경도: 결측치 322개, 결측치 비율 0.27%
주문 내 항목 순번: 결측치 833개, 결측치 비율 0.70%
판매자 ID: 결측치 833개, 결측치 비율 0.70%
배송 마감 기한: 결측치 833개, 결측치 비율 0.70%
상품 가격: 결측치 833개, 결측치 비율 0.70%
배송비: 결측치 833개, 결측치 비율 0.70%
판매자 우편번호 앞자리: 결측치 833개, 결측치 비율 0.70%
판매자 도시: 결측치 833개, 결측치 비율 0.70%
판매자 주: 결측치 833개, 결측치 비율 0.70%
판매자 위도: 결측치 1098개, 결측치 비율 0.92%
판매자 경도: 결측치 1098개, 결측치 비율 0.92%
결제 순번: 결측치 3개, 결측치 비율 0.00%
결제 금액: 결측치 3개, 결측치 비율 0.00%
리뷰 ID: 결측치 997개, 결측치 비율 0.84%
리뷰 점수: 결측치 997개, 결측치 비율 0.84%
리뷰 제목: 결측치 105154개, 결측치 비율 88.26%
리뷰 내용: 결측치 68898개, 결측치 비율 57.83%
리뷰 작성일: 결측치 997개, 결측치 비율 0.84%
리뷰 응답 시각: 결측치 997개, 결측치 비율 0.84

결측치 제거

In [56]:
#결측지 제거
exclude_cols = ['리뷰 점수', '리뷰 제목', '리뷰 내용']
cols_to_check = [col for col in df1.columns if col not in exclude_cols]

# 1. 우선 리뷰 점수/제목/내용 중 하나라도 결측치면 모두 보존
core_na_mask = df1[['리뷰 점수', '리뷰 제목', '리뷰 내용']].isnull().any(axis=1)
core_na_df = df1[core_na_mask]

# 2. 나머지(세 칼럼 다 결측치 아닌 행)만 따로 분리
rest_df = df1[~core_na_mask]

# 3. '리뷰 점수 결측치'인 행 중에서
#   - 제목/내용 결측치 아니고, 나머지에 결측치 있는 행은 제거
#   - 제목/내용 결측치 아니고, 나머지도 정상인 행은 보존
mask_review_score_na = rest_df['리뷰 점수'].isnull()
mask_title_and_content_notna = rest_df['리뷰 제목'].notnull() & rest_df['리뷰 내용'].notnull()
mask_others_no_na = rest_df[cols_to_check].notnull().all(axis=1)

# 조건: (리뷰 점수 결측치) & (제목/내용은 결측치 아니고) & (나머지 결측치 있음)
mask_to_remove = mask_review_score_na & mask_title_and_content_notna & (~mask_others_no_na)

# 해당 행들 제거
final_rest_df = rest_df[~mask_to_remove]

# 4. 두 데이터 합치기 (중복 키는 첫번째 남김)
df2 = pd.concat([core_na_df, final_rest_df]).drop_duplicates(subset=df1.columns[0], keep='first').reset_index(drop=True)

# 5. 결측치 현황 확인
print('최종 리뷰 점수 결측치:', df2['리뷰 점수'].isnull().sum())
print('최종 리뷰 제목 결측치:', df2['리뷰 제목'].isnull().sum())
print('최종 리뷰 내용 결측치:', df2['리뷰 내용'].isnull().sum())

최종 리뷰 점수 결측치: 768
최종 리뷰 제목 결측치: 87891
최종 리뷰 내용 결측치: 58659


In [57]:
df2.isnull().sum()

주문 ID                  0
고객 ID                  0
주문 상태                  0
주문 시간                  0
결제 승인 시각             160
고객 수령일 (실제 배송일)     2965
예상 배송일                 0
고유 고객 ID               0
고객 우편번호 앞자리            0
고객 도시                  0
고객 주(State)            0
고객 우편번호 앞자리          278
고객 위도                278
고객 경도                278
주문 내 항목 순번           775
판매자 ID               775
배송 마감 기한             775
상품 가격                775
배송비                  775
판매자 우편번호 앞자리         775
판매자 도시               775
판매자 주                775
판매자 위도               993
판매자 경도               993
결제 순번                  1
결제 금액                  1
리뷰 ID                768
리뷰 점수                768
리뷰 제목              87891
리뷰 내용              58659
리뷰 작성일               768
리뷰 응답 시각             768
고객 공간 좌표             278
판매자 공간 좌표            993
dtype: int64

In [60]:
df2.dropna(subset=['고객 수령일 (실제 배송일)','결제 승인 시각', '고객 우편번호 앞자리','고객 위도', '고객 경도', '주문 내 항목 순번','판매자 ID', '배송 마감 기한','상품 가격', '배송비', '판매자 우편번호 앞자리','판매자 도시', '판매자 주', '판매자 위도', '판매자 경도', '고객 공간 좌표', '결제 순번', '결제 금액', '판매자 공간 좌표'], inplace=True)

In [61]:
df2.isnull().sum()

주문 ID                  0
고객 ID                  0
주문 상태                  0
주문 시간                  0
결제 승인 시각               0
고객 수령일 (실제 배송일)        0
예상 배송일                 0
고유 고객 ID               0
고객 우편번호 앞자리            0
고객 도시                  0
고객 주(State)            0
고객 우편번호 앞자리            0
고객 위도                  0
고객 경도                  0
주문 내 항목 순번             0
판매자 ID                 0
배송 마감 기한               0
상품 가격                  0
배송비                    0
판매자 우편번호 앞자리           0
판매자 도시                 0
판매자 주                  0
판매자 위도                 0
판매자 경도                 0
결제 순번                  0
결제 금액                  0
리뷰 ID                645
리뷰 점수                645
리뷰 제목              84852
리뷰 내용              57263
리뷰 작성일               645
리뷰 응답 시각             645
고객 공간 좌표               0
판매자 공간 좌표              0
dtype: int64

In [62]:
#결측치 np.nan 대체는 결측치가 있는 위 상태와 동일하다는 지피티의 답변
df2.fillna('없음')

,주문 ID,고객 ID,주문 상태,주문 시간,결제 승인 시각,고객 수령일 (실제 배송일),예상 배송일,고유 고객 ID,고객 우편번호 앞자리,고객 도시,...,결제 순번,결제 금액,리뷰 ID,리뷰 점수,리뷰 제목,리뷰 내용,리뷰 작성일,리뷰 응답 시각,고객 공간 좌표,판매자 공간 좌표
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,...,1.0,18.12,a54f0611adc9ed256b57ede6b6eb5114,4.0,없음,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48,POINT (-46.587161274276774 -23.57698293467452),POINT (-46.444238272702385 -23.680729252163324)
1,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-17 18:06:29,2018-09-04 00:00:00,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,...,1.0,179.12,e73b67b67587f7644d5bd1a52deb1b01,5.0,없음,없음,2018-08-18 00:00:00,2018-08-22 19:07:58,POINT (-48.51478334802312 -16.745149541882594),POINT (-48.22960077888057 -21.36350170979939)
2,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-12-02 00:28:42,2017-12-15 00:00:00,7c142cf63193a1473d2e66489a9ae977,59296,sao goncalo do amarante,...,1.0,72.20,359d03e676b3c069f62cadba8dd3f6e8,5.0,없음,O produto foi exatamente o que eu esperava e e...,2017-12-03 00:00:00,2017-12-05 19:21:58,POINT (-35.271143276096765 -5.774190270584408),POINT (-43.924052542673515 -19.83768155726055)
3,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-16 18:17:02,2018-02-26 00:00:00,72632f0f9dd73dfee390c9b22eb56dd6,9195,santo andre,...,1.0,28.62,e50934924e227544ba8246aeb3770dd4,5.0,없음,없음,2018-02-17 00:00:00,2018-02-18 13:02:51,POINT (-46.514627329509395 -23.67636972794609),POINT (-46.262085681846166 -23.543394600163566)
4,a4591c265e18cb1dcee52889e2d8acc3,503740e9ca751ccdda7ba28e9ab8f608,delivered,2017-07-09 21:57:05,2017-07-09 22:10:13,2017-07-26 10:57:55,2017-08-01 00:00:00,80bb27c7c16e8f973207a5086ab329e2,86320,congonhinhas,...,1.0,175.26,89b738e70a1ce346db29a20fb2910161,4.0,없음,없음,2017-07-27 00:00:00,2017-07-27 22:48:30,POINT (-50.54992367333536 -23.553522043896585),POINT (-46.516141894926825 -23.46870426001278)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99436,c7510fe2bce3ab54854dc588ce4d187c,f74562da630b57572d1742ba2f28b662,delivered,2018-05-16 09:05:27,2018-05-17 09:18:23,2018-05-22 14:20:48,2018-06-21 00:00:00,5112d7286a328a07d06a8051c3c55410,21616,rio de janeiro,...,1.0,220.12,bbe7c6000965c0d7b0a0fbd9c9fa7b24,5.0,excelente,"nota 1000 , Gostei muito do produto , material...",2018-05-23 00:00:00,2018-05-26 13:43:20,POINT (-43.396809432222284 -22.85387973916863),POINT (-51.467963988245124 -29.68780422662315)
99437,52018484704db3661b98ce838612b507,e450a297a7bc6839ceb0cf1a2377fa02,delivered,2018-08-29 12:25:59,2018-08-29 12:35:17,2018-08-30 22:48:27,2018-09-03 00:00:00,7a22d14aa3c3599238509ddca4b93b01,5863,sao paulo,...,1.0,73.10,7a11bf826668febba0800ec35884958c,1.0,Muito frágil !!!,"Achei o produto muito pequeno e onde fica a ""t...",2018-08-31 00:00:00,2018-09-21 13:54:38,POINT (-46.76618596319816 -23.676918147471813),POINT (-46.73872785910907 -23.502723037382403)
99438,a6bd1f93b7ff72cc348ca07f38ec4bee,6d63fa86bd2f62908ad328325799152f,delivered,2018-04-20 17:28:40,2018-04-24 19:26:10,2018-04-28 17:38:42,2018-05-15 00:00:00,9108b540419f20edc605468f3966813b,15093,sao jose do rio preto,...,1.0,77.23,eb4c94cf604d9cf4d4442890ac33797d,5.0,recomendo,Foi tudo dentro do esperado!!!,2018-04-29 00:00:00,2018-05-07 13:31:59,POINT (-49.410999511658055 -20.855380218943672),POINT (-46.34087492614052 -23.52194421054593)
99439,5597332b7eded552f104108f22b023e4,aaa423fb52f4106f477683490cbd5845,delivered,2018-08-15 13:03:37,2018-08-15 13:15:22,2018-08-17 16:06:37,2018-08-21 00:00:00,8a898880a61e551c80bacadfb4356255,6449,barueri,...,1.0,36.46,c44004d7e60dc281ebd5361b717570c3,5.0,Ótima,Mui

In [63]:
#중복값
df2.duplicated().sum()

np.int64(0)

In [65]:
df2.columns
cols = df2.columns
dupe_idx = [i for i, x in enumerate(cols) if x == '고객 우편번호 앞자리']
print(dupe_idx)
mask = [True if i != dupe_idx[1] else False for i in range(len(df2.columns))]
df2 = df2.loc[:, mask]

print(df2.columns)

[8, 11]
Index(['주문 ID', '고객 ID', '주문 상태', '주문 시간', '결제 승인 시각', '고객 수령일 (실제 배송일)',
       '예상 배송일', '고유 고객 ID', '고객 우편번호 앞자리', '고객 도시', '고객 주(State)', '고객 위도',
       '고객 경도', '주문 내 항목 순번', '판매자 ID', '배송 마감 기한', '상품 가격', '배송비',
       '판매자 우편번호 앞자리', '판매자 도시', '판매자 주', '판매자 위도', '판매자 경도', '결제 순번', '결제 금액',
       '리뷰 ID', '리뷰 점수', '리뷰 제목', '리뷰 내용', '리뷰 작성일', '리뷰 응답 시각', '고객 공간 좌표',
       '판매자 공간 좌표'],
      dtype='object')


In [67]:
import pandas as pd

# 컬럼명 공백 제거 (에러 방지)
df2.columns = df2.columns.str.strip()

# 한글 깨짐 없이, 행 번호까지 저장
df2.to_csv('cleand_data_최종.csv', index=True, encoding='utf-8-sig', index_label='행번호')